In [101]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, AvgPool2D, Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
RANDOM_SEED = 12345

datagen_train = ImageDataGenerator(rescale=1./255)

datagen_flow_train = datagen_train.flow_from_dataframe(
    dataframe='/datasets/faces/labels.csv',
    directory='/datasets/faces',
    x_col='file_name',
    y_col='real_age',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    seed=RANDOM_SEED
)

features_train, target_train = next(datagen_flow_train)

fig = plt.figure(figsize=(10, 10))

for i in range(15):
    fig.add_subplot(4, 4, i+1)
    plt.imshow(features_train[i])
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()

In [105]:
def load_train(path):
    labels = pd.read_csv(path + 'labels.csv')

    train_datagen = ImageDataGenerator(
    validation_split=0.25,
    rescale=1./255,
    horizontal_flip=True
    )

    train_datagen_flow = train_datagen.flow_from_dataframe(
    dataframe=labels,
    directory=path + 'final_files/',
    x_col='file_name',
    y_col='real_age',
    target_size=(224, 224),
    batch_size=16,
    class_mode='raw',
    subset='training',
    seed=RANDOM_SEED)

    return train_datagen_flow

In [106]:
def load_test(path):
    labels = pd.read_csv(path + 'labels.csv')

    validation_datagen = ImageDataGenerator(validation_split=0.25, rescale=1./255)
    val_datagen_flow = validation_datagen.flow_from_dataframe(
    dataframe=labels,
    directory=path + 'final_files/',
    x_col='file_name',
    y_col='real_age',
    target_size=(224, 224),
    batch_size=16,
    class_mode='raw',
    subset='validation',
    seed=RANDOM_SEED)

    return val_datagen_flow

In [112]:
def create_model(input_shape):
    backbone = ResNet50(
    input_shape=input_shape,
    weights='imagenet',
    include_top=False
    )

    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mae', optimizer=Adam(lr=0.0001), metrics=['mae'])
    return model

In [110]:
def train_model(model, train_data, test_data, batch_size=None, epochs=10,
               steps_per_epoch=None, validation_steps=None):

    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data) 
    
    model.fit(
        train_data,
        validation_data=test_data,
        batch_size=batch_size,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        verbose=2,
        shuffle=True
    )
    return model

In [113]:
if __name__ == '__main__':
    train_data = load_train('/datasets/faces/')
    test_data = load_test('/datasets/faces/')
    model = create_model(train_data.shape[1:])
    model = train_model(model, train_data, test_data)
    loss, mae = model.evaluate(test_data[0], test_data[1], verbose=2)
    print('Model mae', mae)